## A demo using Hydrogen Hamiltonian with GPT-QE.

In [1]:
import torch
from gqe.mingpt.utils import set_seed
seed = 3407
set_seed(seed)

In [2]:
from qwrapper.operator import PauliObservable
from gqe.mingpt.cost import EnergyCost
from qswift.compiler import DefaultOperatorPool
from benchmark.molecule import DiatomicMolecularHamiltonian
from gqe.operator_pool.uccsd import UCCSD, do_generate_molecule
from gqe.common.initializer import HFStateInitializer
from gqe.util import get_device
from gqe.mingpt.callback import DefaultCallback, PrintMonitor, FileMonitor

# molecule = generate_molecule("Li", "H", 1.596, "sto-3g", bravyi_kitaev=False)
bond_length = 3.0
geometry = f"H 0.0 0.0 0.0\n" + f"Be 0.0 0.0 {bond_length}\n" + f"H 0.0 0.0 {2 * bond_length}\n"
molecule = do_generate_molecule(geometry, "sto-3g", bravyi_kitaev=False)
nqubit = 12

# prepare Hamiltonian
hamiltonian = DiatomicMolecularHamiltonian(nqubit, molecule, bravyi_kitaev=False)

# prepare operator_pool
uccsd = UCCSD(nqubit, molecule)
paulis = uccsd.paulis
paulis.append(PauliObservable("IIIIIIIIII"))
print('paulis', paulis)
num_operators = len(paulis)
initializer = HFStateInitializer(n_electrons=4)
pool = DefaultOperatorPool(paulis)
# time_pool = [1 / 320, -1 / 320, 1 / 160, -1 / 160, 1 / 80, -1 / 80, 1 / 40, -1 / 40, 0.05, -0.05, 0.1, -0.1, 0.2, -0.2]
time_pool = [1 / (2 ** j) for j in range(2, 12)]
time_pool.extend([-1 / (2 ** j) for j in range(2, 12)])
cost = EnergyCost(hamiltonian, initializer, pool, time_pool)


converged SCF energy = -15.0242100060364
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
paulis [+IIIYZXIIIIII, +IIIYZZZXIIII, +IIIYZZZZZXII, +IIIYZZZZZZZX, +IIXXIIIIIIYX, +IIXXIIIIYXII, +IIXXIIYXIIII, +IIXXYXIIIIII,

## FCI energy by diagonalization

In [3]:
from qwrapper.hamiltonian import compute_ground_state

# print(compute_ground_state(hamiltonian))

In [4]:
print("hf state:", hamiltonian.exact_value(initializer.init_circuit(12, [], "qulacs")))

hf state: -15.024210006036476


## Setup for GPT

In [5]:
# create a GPT instance
from gqe.mingpt.model import GPT

num_layers = 1



In [6]:
# create a Trainer object
from gqe.mingpt.trainer import Trainer
from gqe.mingpt.layer import LayerWiseTrainer, LayerWiseFactory

pool = DefaultOperatorPool(paulis)

# time_pool = [1 / 320, -1 / 320, 1 / 160, -1 / 160, 1 / 80, -1 / 80, 1 / 40, -1 / 40, 0.05, -0.05, 0.1, -0.1, 0.2, -0.2]
import math

time_pool = [math.pi / 2 ** j for j in range(3, 10)]
time_pool.extend([-math.pi / 2 ** j for j in range(3, 10)])

cost = EnergyCost(hamiltonian, initializer, pool, time_pool)


class CustomFactory(LayerWiseFactory):
    def generate_train_config(self, layer):
        train_config = Trainer.get_default_config()
        train_config.learning_rate = 5e-7  # the model we're using is so small that we can go a bit faster
        train_config.max_iters = 500
        train_config.num_workers = 5
        train_config.n_samples = 50
        return train_config

    def generate_model_config(self, layer):
        model_config = GPT.get_default_config()
        model_config.model_type = 'gpt2'
        model_config.vocab_size = self.generate_cost(layer).vocab_size()
        model_config.n_gates = 10  # The number of gates for each circuit
        model_config.block_size = model_config.n_gates
        model_config.temperature = 5  # Each gate is generated with probability exp(-temperature * logit)
        model_config.embd_pdrop = 0.1
        model_config.resid_pdrop = 0.1
        model_config.attn_pdrop = 0.1
        model_config.std = 0.02
        model_config.energy_offset = 14
        return model_config

    def generate_cost(self, layer):
        return cost


trainer = LayerWiseTrainer(CustomFactory(), cost, num_layers, get_device())

In [7]:
print_monitors = []
file_monitors = []
for index in range(num_layers):
    file_monitor = FileMonitor()
    file_monitors.append(file_monitor)
    trainer.set_monitors(index, [PrintMonitor(), file_monitor])
trainer.run()
#torch.save(model.state_dict(), '../saved_models/gptqe_test_2')

layer: 1 starts running
number of parameters: 85.89M
running on device mps
iter_dt 0.00s; iter 0: train loss 0.28407 temperature: 5
mean_logits tensor([-14.8615, -14.8464, -15.1294, -14.8281, -14.7885, -14.7122, -14.8551,
        -15.0072, -14.7934, -14.9684, -14.9633, -14.8020, -14.9232, -14.6650,
        -14.6780, -14.7429, -14.7700, -14.9381, -15.0550, -14.8812, -14.7757,
        -14.9613, -15.0290, -14.8795, -14.8206, -14.7417, -14.8812, -14.8917,
        -14.8877, -14.8142, -14.8543, -14.7515, -15.2146, -14.8133, -14.6951,
        -14.7373, -14.6050, -14.7226, -14.8874, -14.7764, -14.7126, -14.9584,
        -15.1361, -15.0330, -15.1672, -14.8113, -15.0958, -14.6992, -14.8126,
        -14.8776], device='mps:0', grad_fn=<SubBackward0>)
energies: tensor([-14.7757, -14.9635, -14.2763, -14.7723, -14.9859, -14.9840, -14.7213,
        -14.6537, -14.7117, -14.8687, -14.8315, -15.0198, -14.8143, -15.1219,
        -14.8771, -14.9108, -14.7117, -14.9877, -14.8945, -14.4468, -14.8337,
       


KeyboardInterrupt



In [ ]:
indices = []
for fm in file_monitors:
    print(fm.min_energy)
    indices.extend(fm.min_indices)



In [ ]:
from gqe.util import to_pqc, to_time_evolutions
from gqe.vqa.initializer import InitializerDelegate
from qwrapper.optimizer import AdamOptimizer
from qml.core.vqe import VQE
from qml.core.pqc import TimeEvolutionPQC
from qml.core.function import Energy

pqc = TimeEvolutionPQC(cost.sequence.observable.nqubit)
evolutions = to_time_evolutions(cost.sequence, file_monitors[0].min_indices)
for evolution in evolutions:
    pqc.add_time_evolution(evolution.pauli, evolution.t)
initializer = InitializerDelegate(HFStateInitializer(n_electrons=4), nqubit, "qulacs")
optimizer = AdamOptimizer(maxiter=3000)

energy = Energy(hamiltonian, nqubit, pqc)
vqe = VQE(energy, initializer, optimizer)
print(vqe.value())
vqe.exec()

print(vqe.value())
# cost.sequence.tool = "qiskit"
# print(file_monitor.min_indices)
# cost.sequence._get_circuit(file_monitor.min_indices).qc.draw(output="mpl", plot_barriers=True)